In [1]:
import os

# Restrict PyTorch to only see GPU 0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch

if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available, using CPU.")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Number of GPUs available: 1
GPU 0: NVIDIA L40S
Using device: cuda:0


In [2]:
from spatialvla.mobilevlm.model.diffusion_heads import DiffusionActionHead
import torch
import torch.optim as optim
from tqdm import tqdm
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import wandb
from einops import rearrange
from spatialvla.mobilevlm.model.diffusion_heads import DiffusionActionHead, FourierFeatures
DiffusionHead = {
    'head_type': 'Diffusion',
    'hidden_projection': 'pass',
    'use_map' : True,
    'max_action': 5.0,
    'loss_type': 'mse',
    'time_dim':  32,
    'num_blocks': 3,
    'dropout_rate': 0.0,
    'hidden_dim': 1024,
    'use_layer_norm':True,
    'diffusion_steps': 100,
    'n_diffusion_samples': 1,
}

In [3]:
model = DiffusionActionHead(in_dim=32, action_dim=28*28, head_args=DiffusionHead).cuda()

In [4]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params/1000000}million")

Total number of parameters: 31.075584million


In [5]:
import numpy as np
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Normalize to [-1, 1] for MNIST
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the image to (784,)
])
batch_size = 2048
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [6]:
epochs = 1000
learning_rate = 1e-4
from transformers import get_cosine_schedule_with_warmup

# Assuming `optimizer` is defined

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.1)
fourier_encoder = FourierFeatures(output_size=32, learnable=False).cuda()
num_warmup_ratio = 0.06  # Adjust based on model size and data
num_training_steps = int(epochs * len(train_loader))
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(num_training_steps * num_warmup_ratio),
    num_training_steps=num_training_steps
)

2024-11-14 21:00:22.248992: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-14 21:00:22.276999: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-14 21:00:22.277041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-14 21:00:22.277787: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 21:00:22.282542: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
model.train()
# Training loop
for epoch in range(epochs):
    model.train()     
    optimizer.zero_grad()
    for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")):
        # with torch.autocast(device_type='cuda', dtype=torch.float16):
        images = images.cuda()
        labels = labels.cuda()
        labels_encoded = fourier_encoder(labels.float().view(-1, 1, 1))
        loss = model.loss(labels_encoded, images.unsqueeze(1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
    current_lr = scheduler.get_last_lr()[0]
    print(f"Current learning rate = {current_lr}")
    print(loss)

Epoch 1/1000: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:06<00:00,  4.89it/s]


Current learning rate = 1.6666666666666667e-06
tensor(1.0156, device='cuda:0', grad_fn=<MeanBackward0>)


Epoch 2/1000: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:05<00:00,  5.33it/s]


Current learning rate = 3.3333333333333333e-06
tensor(1.0061, device='cuda:0', grad_fn=<MeanBackward0>)


Epoch 3/1000: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:05<00:00,  5.04it/s]


...

Epoch 585/1000: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:05<00:00,  5.19it/s]


Current learning rate = 4.0860840509215496e-05
tensor(0.1772, device='cuda:0', grad_fn=<MeanBackward0>)


Epoch 586/1000: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:05<00:00,  5.21it/s]


Current learning rate = 4.069660105744469e-05
tensor(0.1789, device='cuda:0', grad_fn=<MeanBackward0>)


Epoch 587/1000:  93%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋        | 28/30 [00:05<00:00,  5.26it/s]

In [ ]:
loss.item()

In [ ]:
p, h = model.predict_action(fourier_encoder(torch.Tensor([[8], [8]]).view(-1, 1).cuda()), return_history=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
for image in p:
    image = (image.reshape(28, 28).detach() * 0.5 + 0.5).clamp(0, 1).cpu().numpy()
    img = Image.fromarray(image*255.0)
    plt.imshow(img)
    break

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Assuming `h` is a list of tensors
frames = []

# Loop through each element in `h` to create and save each frame
for h_elem in h:
    # Plot the data
    fig, ax = plt.subplots()
    ax.plot(range(7), h_elem[0].detach().cpu().numpy())

    # Save the plot as an image in memory
    fig.canvas.draw()
    image = np.array(fig.canvas.renderer.buffer_rgba())
    frames.append(Image.fromarray(image))

    # Close the figure to avoid display overlap and memory issues
    plt.close(fig)

# Save frames as an animated GIF
frames[0].save("animation.gif", save_all=True, append_images=frames[1:], loop=0, duration=200)
